# Notebook to format sample layout with annotated expected mutants

This notebook takes as input a dataframe containing all expected mutants for each mutated locus. It annotates said mutants and merges the result to the sample layout.

## Import libraries

In [1]:
import pandas as pd

## Specify paths

In [2]:
# Note, I used to import config entries as "params" (i.e. listed in the params statement of the rule), but...
# a bug led to params not being imported.. (snakemake object is imported but not the params attribute)
# Therefore, now I will import all config entries directly from the snakemake object
layout_path = (
    snakemake.input.layout  # Projet-specific file containing the sample layout
)
exp_mut_path = (
    snakemake.input.exp_mut  # Projet-specific file containing the expected mutated sequences
)
codon_table_path = snakemake.config["codon"][
    "table"  # Projet-specific file containing the genetic code
]
output_file = snakemake.output[0]

## Import codon table

In [3]:
codon_table = pd.read_csv(codon_table_path, header=0)
codon_table["codon"] = codon_table["codon"].str.upper()
codon_table.head(3)

,codon,aminoacid,freq,number
0,TTT,F,26.26,76999
1,TTC,F,17.89,52459
2,TTA,L,26.31,77131


In [4]:
# Convert to dictionary
codon_dic = dict(zip(codon_table["codon"], codon_table["aminoacid"]))

## Define and test functions

In [5]:
def get_aa_seq(seq, codon_dic):

    if len(seq) % 3 != 0:
        raise Exception(
            f"Error.. the length of the provided wild-type DNA sequence is not a multiple of 3."
        )

    for x in set(seq):
        if x not in ["A", "C", "G", "T"]:
            raise Exception(
                f"Error.. one of the provided nucleotide sequences contains an unrecognized character."
            )

    clist = [
        seq[i : i + 3]
        for i in range(0, len(seq), 3)  # Convert nucleotide sequence to list of codons
    ]

    return "".join([codon_dic[x] for x in clist])

In [6]:
get_aa_seq("TTCCCTTAA", codon_dic)

'FP*'

In [7]:
def get_mutations(seq, wt, codon_dict):
    """
    By comparing a mutated DNA sequence to the wild-type sequence,
    this function returns the mutations (if there are any).
    Mutations are formatted as # mutated codon / position / alternative codon / alternative amino acid
    in lists with matching indexes to be able to quickly convert to 1 row per mutation per mutated codon
    The alternative and corresponding wild-type codons are translated into their corresponding amino acid using the provided codon table dictionary
    From there, we also calculate the Hamming distances in codons, nucleotides and amino acids.
    """
    if len(seq) != len(wt):
        raise Exception(
            f"Error.. Cannot annotate expected mutants because at least one sequence is of different length than wild-type."
        )

    is_wt = seq == wt

    list_alt_pos, list_alt_cod, list_alt_aa = [], [], []
    Nham_nt = 0
    Nham_aa = 0

    wt_list_codons = [
        wt[i : i + 3]
        for i in range(
            0, len(wt), 3
        )  # Converting WT nucleotide sequence to list of codons
    ]
    seq_list_codons = [
        seq[i : i + 3]
        for i in range(
            0, len(seq), 3
        )  # Converting nucleotide sequence of variant to list of codons
    ]

    for i, c in enumerate(seq_list_codons):  # Loop through codons
        wtc = wt_list_codons[i]  # Retrieve wild-type codon
        if c != wtc:
            list_alt_pos.append(i)
            list_alt_cod.append(c)
            list_alt_aa.append(codon_dict[c])
            Nham_nt += sum(1 for x, y in zip(wtc, c) if x != y)
            if codon_dict[c] != codon_dict[wtc]:
                Nham_aa += 1

    Nham_codons = len(list_alt_pos)

    if Nham_codons > 0:
        mut_codons = list(range(1, len(list_alt_pos) + 1))
    else:
        mut_codons = [0]
        list_alt_pos = ["not-applicable"]
        list_alt_cod = ["not-applicable"]
        list_alt_aa = ["not-applicable"]

    return (
        is_wt,
        Nham_codons,
        Nham_nt,
        Nham_aa,
        mut_codons,
        list_alt_pos,
        list_alt_cod,
        list_alt_aa,
    )

In [8]:
get_mutations("TTCCCTTTA", "TTCCTATTA", codon_dic)

(False, 1, 2, 1, [1], [1], ['CCT'], ['P'])

## Import expected mutants

In [9]:
exp_mut = pd.read_csv(exp_mut_path)
for x in [y for y in exp_mut.columns if y in ["WT_seq", "nt_seq", "barcode"]]:
    exp_mut[x] = exp_mut[x].str.upper()
exp_mut

,Mutated_seq,WT_seq,barcode,nt_seq
0,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCAATCCTGTAGTCGCCTTCTGTACTCA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCTCTGTTTGTT...
1,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGTAGTCTGTGACCCTAGGATGCCACATT,GCCAAGGAAACCATATACGTGATGTTAAGGTTTTTCGACGTTTGTT...
2,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCGATCTTATCGGAGTTCCATGTCCATC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...
3,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCGTTACTGTCTCATTACGCCTAGTTCGT,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTG...
4,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGCCTCATTGACTGTGTGCGTATAACGTG,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACTCTTGTT...
...,...,...,...,...
8472,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCACTTGCCATGAGGCTAATGTGGCCTAA,CCTAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...
8473,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGGCAATCCTGTAGGTGCCTTCTGTACTGA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTACGT...
8474,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,GATACGAGTCGGATGTAGAGGAGAGACATT,GCCAAGGAAACCATATACATCATGTTAAGGTGGTTCGACGTTTGTT...
8475,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGTGGAGTCCAAGAGTCGAACTAGGAACGC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...


## Translate nucleotide sequences into amino acid sequences

In [10]:
exp_mut["aa_seq"] = exp_mut.nt_seq.apply(lambda x: get_aa_seq(x, codon_dic))

## Get mutations by comparing sequences to WT

In [11]:
per_seq_cols = ["WT", "Nham_codons", "Nham_nt", "Nham_aa"]
per_mut_cols = ["mutated_codon", "pos", "alt_codons", "alt_aa"]
new_cols = per_seq_cols + per_mut_cols
collected_mutations = exp_mut.apply(
    lambda row: get_mutations(row.nt_seq, row.WT_seq, codon_dic), axis=1
)
mutations_dict = dict(zip(new_cols, zip(*collected_mutations)))
exp_mut = exp_mut.assign(**mutations_dict)
exp_mut

,Mutated_seq,WT_seq,barcode,nt_seq,aa_seq,WT,Nham_codons,Nham_nt,Nham_aa,mutated_codon,pos,alt_codons,alt_aa
0,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCAATCCTGTAGTCGCCTTCTGTACTCA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCTCTGTTTGTT...,AKETIYIMLRFFSVCFHHINQGCVS,False,1,3,1,[1],[12],[TCT],[S]
1,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGTAGTCTGTGACCCTAGGATGCCACATT,GCCAAGGAAACCATATACGTGATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYVMLRFFDVCFHHINQGCVS,False,1,2,1,[1],[6],[GTG],[V]
2,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCGATCTTATCGGAGTTCCATGTCCATC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHCNQGCVS,False,1,3,1,[1],[18],[TGT],[C]
3,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCGTTACTGTCTCATTACGCCTAGTTCGT,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTG...,AKETIYIMLRFFDVCVHHINQGCVS,False,1,2,1,[1],[15],[GTG],[V]
4,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGCCTCATTGACTGTGTGCGTATAACGTG,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACTCTTGTT...,AKETIYIMLRFFDSCFHHINQGCVS,False,1,2,1,[1],[13],[TCT],[S]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8472,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCACTTGCCATGAGGCTAATGTGGCCTAA,CCTAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,PKETIYIMLRFFDVCFHHINQGCVS,False,1,2,1,[1],[0],[CCT],[P]
8473,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGGCAATCCTGTAGGTGCCTTCTGTACTGA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTACGT...,AKETIYIMLRFFDVTFHHINQGCVS,False,1,3,1,[1],[14],[ACG],[T]
8474,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,GATACGAGTCGGATGTAGAGGAGAGACATT,GCCAAGGAAACCATATACATCATGTTAAGGTGGTTCGACGTTTGTT...,AKETIYIMLRWFDVCFHHINQGCVS,False,1,2,1,[1],[10],[TGG],[W]
8475,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGTGGAGTCCAAGAGTCGAACTAGGAACGC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHINQGCVS,True,0,0,0,[0],[not-applicable],[not-applicable],[not-applicable]


In [12]:
exp_mut_long = exp_mut.explode(per_mut_cols).reset_index(drop=True)
exp_mut_long

,Mutated_seq,WT_seq,barcode,nt_seq,aa_seq,WT,Nham_codons,Nham_nt,Nham_aa,mutated_codon,pos,alt_codons,alt_aa
0,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCAATCCTGTAGTCGCCTTCTGTACTCA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCTCTGTTTGTT...,AKETIYIMLRFFSVCFHHINQGCVS,False,1,3,1,1,12,TCT,S
1,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGTAGTCTGTGACCCTAGGATGCCACATT,GCCAAGGAAACCATATACGTGATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYVMLRFFDVCFHHINQGCVS,False,1,2,1,1,6,GTG,V
2,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,CGGCGATCTTATCGGAGTTCCATGTCCATC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHCNQGCVS,False,1,3,1,1,18,TGT,C
3,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCGTTACTGTCTCATTACGCCTAGTTCGT,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTG...,AKETIYIMLRFFDVCVHHINQGCVS,False,1,2,1,1,15,GTG,V
4,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AAGCCTCATTGACTGTGTGCGTATAACGTG,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACTCTTGTT...,AKETIYIMLRFFDSCFHHINQGCVS,False,1,2,1,1,13,TCT,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8472,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,TCCACTTGCCATGAGGCTAATGTGGCCTAA,CCTAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,PKETIYIMLRFFDVCFHHINQGCVS,False,1,2,1,1,0,CCT,P
8473,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGGCAATCCTGTAGGTGCCTTCTGTACTGA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTACGT...,AKETIYIMLRFFDVTFHHINQGCVS,False,1,3,1,1,14,ACG,T
8474,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,GATACGAGTCGGATGTAGAGGAGAGACATT,GCCAAGGAAACCATATACATCATGTTAAGGTGGTTCGACGTTTGTT...,AKETIYIMLRWFDVCFHHINQGCVS,False,1,2,1,1,10,TGG,W
8475,F13,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AGTGGAGTCCAAGAGTCGAACTAGGAACGC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHINQGCVS,True,0,0,0,0,not-applicable,not-applicable,not-applicable


## Import layout

In [13]:
layout = pd.read_csv(layout_path)
layout

,Sample_name,R1,R2,N_forward,N_reverse,Pos_start,Pos_stop,Replicate,Fragment,Timepoint,Mutated_seq,Species,Drug
0,Barc_T0_A_CTL,Barc_T0_A_R1.fastq.gz,Barc_T0_A_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,A,F13,T0,F13,SC,CTL
1,Barc_T0_B_CTL,Barc_T0_B_R1.fastq.gz,Barc_T0_B_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,B,F13,T0,F13,SC,CTL
2,Barc_T0_C_CTL,Barc_T0_C_R1.fastq.gz,Barc_T0_C_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,C,F13,T0,F13,SC,CTL
3,Barc_T1_A_CTL,Barc_T1_A_CTL_R1.fastq.gz,Barc_T1_A_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,A,F13,T1,F13,SC,CTL
4,Barc_T1_B_CTL,Barc_T1_B_CTL_R1.fastq.gz,Barc_T1_B_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,B,F13,T1,F13,SC,CTL
5,Barc_T1_C_CTL,Barc_T1_C_CTL_R1.fastq.gz,Barc_T1_C_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,C,F13,T1,F13,SC,CTL
6,Barc_T2_A_CTL,Barc_T2_A_CTL_R1.fastq.gz,Barc_T2_A_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,A,F13,T2,F13,SC,CTL
7,Barc_T2_B_CTL,Barc_T2_B_CTL_R1.fastq.gz,Barc_T2_B_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,B,F13,T2,F13,SC,CTL
8,Barc_T2_C_CTL,Barc_T2_C_CTL_R1.fastq.gz,Barc_T2_C_CTL_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,C,F13,T2,F13,SC,CTL
9,Barc_T0_A_POSA,Barc_T0_A_R1.fastq.gz,Barc_T0_A_R2.fastq.gz,'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG;optional......,'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG;optional.....,301,326,A,F13,T0,F13,SC,POSA


## Merge onto layout

In [14]:
withSeqs = layout.drop(["R1", "R2", "N_forward", "N_reverse"], axis=1).merge(
    right=exp_mut_long.drop("WT_seq", axis=1), on="Mutated_seq"
)
withSeqs

,Sample_name,Pos_start,Pos_stop,Replicate,Fragment,Timepoint,Mutated_seq,Species,Drug,barcode,nt_seq,aa_seq,WT,Nham_codons,Nham_nt,Nham_aa,mutated_codon,pos,alt_codons,alt_aa
0,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,CGGCAATCCTGTAGTCGCCTTCTGTACTCA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCTCTGTTTGTT...,AKETIYIMLRFFSVCFHHINQGCVS,False,1,3,1,1,12,TCT,S
1,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,AAGTAGTCTGTGACCCTAGGATGCCACATT,GCCAAGGAAACCATATACGTGATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYVMLRFFDVCFHHINQGCVS,False,1,2,1,1,6,GTG,V
2,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,CGGCGATCTTATCGGAGTTCCATGTCCATC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHCNQGCVS,False,1,3,1,1,18,TGT,C
3,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,TCCGTTACTGTCTCATTACGCCTAGTTCGT,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTG...,AKETIYIMLRFFDVCVHHINQGCVS,False,1,2,1,1,15,GTG,V
4,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,AAGCCTCATTGACTGTGTGCGTATAACGTG,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACTCTTGTT...,AKETIYIMLRFFDSCFHHINQGCVS,False,1,2,1,1,13,TCT,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228874,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,TCCACTTGCCATGAGGCTAATGTGGCCTAA,CCTAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,PKETIYIMLRFFDVCFHHINQGCVS,False,1,2,1,1,0,CCT,P
228875,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,AGGCAATCCTGTAGGTGCCTTCTGTACTGA,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTACGT...,AKETIYIMLRFFDVTFHHINQGCVS,False,1,3,1,1,14,ACG,T
228876,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,GATACGAGTCGGATGTAGAGGAGAGACATT,GCCAAGGAAACCATATACATCATGTTAAGGTGGTTCGACGTTTGTT...,AKETIYIMLRWFDVCFHHINQGCVS,False,1,2,1,1,10,TGG,W
228877,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,AGTGGAGTCCAAGAGTCGAACTAGGAACGC,GCCAAGGAAACCATATACATCATGTTAAGGTTTTTCGACGTTTGTT...,AKETIYIMLRFFDVCFHHINQGCVS,True,0,0,0,0,not-applicable,not-applicable,not-applicable


## Add position offset

In [16]:
withSeqs["aa_pos"] = withSeqs.apply(
    lambda row: (
        row.pos + row.Pos_start if row.pos != "not-applicable" else "not-applicable"
    ),
    axis=1,
)
withSeqs

,Sample_name,Pos_start,Pos_stop,Replicate,Fragment,Timepoint,Mutated_seq,Species,Drug,barcode,...,aa_seq,WT,Nham_codons,Nham_nt,Nham_aa,mutated_codon,pos,alt_codons,alt_aa,aa_pos
0,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,CGGCAATCCTGTAGTCGCCTTCTGTACTCA,...,AKETIYIMLRFFSVCFHHINQGCVS,False,1,3,1,1,12,TCT,S,313
1,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,AAGTAGTCTGTGACCCTAGGATGCCACATT,...,AKETIYVMLRFFDVCFHHINQGCVS,False,1,2,1,1,6,GTG,V,307
2,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,CGGCGATCTTATCGGAGTTCCATGTCCATC,...,AKETIYIMLRFFDVCFHHCNQGCVS,False,1,3,1,1,18,TGT,C,319
3,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,TCCGTTACTGTCTCATTACGCCTAGTTCGT,...,AKETIYIMLRFFDVCVHHINQGCVS,False,1,2,1,1,15,GTG,V,316
4,Barc_T0_A_CTL,301,326,A,F13,T0,F13,SC,CTL,AAGCCTCATTGACTGTGTGCGTATAACGTG,...,AKETIYIMLRFFDSCFHHINQGCVS,False,1,2,1,1,13,TCT,S,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228874,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,TCCACTTGCCATGAGGCTAATGTGGCCTAA,...,PKETIYIMLRFFDVCFHHINQGCVS,False,1,2,1,1,0,CCT,P,301
228875,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,AGGCAATCCTGTAGGTGCCTTCTGTACTGA,...,AKETIYIMLRFFDVTFHHINQGCVS,False,1,3,1,1,14,ACG,T,315
228876,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,GATACGAGTCGGATGTAGAGGAGAGACATT,...,AKETIYIMLRWFDVCFHHINQGCVS,False,1,2,1,1,10,TGG,W,311
228877,Barc_T2_C_NQO,301,326,C,F13,T2,F13,SC,NQO,AGTGGAGTCCAAGAGTCGAACTAGGAACGC,...,AKETIYIMLRFFDVCFHHINQGCVS,True,0,0,0,0,not-applicable,not-applicable,not-applicable,not-applicable


In [18]:
withSeqs.to_csv(output_file)